# Final Exam Assignment (40 points - Total of 45 is Possible)
## Due December 7, 2022, @ 8:00 am
Note that there will be no extensions given for this assignment as there is a tight timeline for grading. 

For this assignment, I have provided each of you with your own training dataset. Your goal is to train a deep neural network to uncover the code image provided to you. 

I will provide you with instructions throughout. 

In [1]:
import os

from urllib.request import urlopen
from urllib.parse import urlencode
import urllib.request

import time
import sys

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, train_test_split

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from torch.utils.data import Dataset
from torchvision import datasets


In [2]:
# This is a tool I have provided you to help you download your file.

def download_file(url, filename):
    """
    A function that downloads the data file from a URL
    Parameters
    ----------
    url : string
        url where the file to download is located
    filename : string
        location where to save the file
    reporthook : function
        callback to display the download progress
    """
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(url, filename, reporthook)
        
def reporthook(count, block_size, total_size):
    """
    A function that displays the status and speed of the download
    """

    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = int(progress_size / (1024 * duration + 0.0001))
    percent = int(count * block_size * 100 / total_size)
    sys.stdout.write("\r...%d%%, %d MB, %d KB/s, %d seconds passed" %
                     (percent, progress_size / (1024 * 1024), speed, duration))
    sys.stdout.flush()

In [3]:
# You can download your file by typing your first name into the name block
# The name used is the first part of your first name as listed in BB learn
# If you have problems downloading the data please reach out to me

name = 'Engin'
download_file(f'https://zenodo.org/record/7339649/files/data_{name}.npz?download=1','data.npz')

## Loading the Data (3 points)
The data is provided to you as a compressed NumPy array saved as 'data.npz'. When working with real data you might need to figure out how data is stored. Use the information on 'npz' files to figure out what data you have. The data file contains three NumPy arrays. 
1. The features for the training dataset
2. The regression values for the training dataset
3. The validation features that contain your code

In [4]:
df=np.load("data.npz")
df.files

['training_feat', 'training_true', 'validation_feat']

In [5]:
with np.load('data.npz') as data:
    training_feat = data['training_feat']
    training_true = data['training_true']
    validation_feat = data['validation_feat']

In [6]:
print(f"training_feat:{training_feat.shape}\ntraining_true:{training_true.shape} \nvalidation_feat:{validation_feat.shape}")

training_feat:(100000, 30)
training_true:(100000, 3) 
validation_feat:(65536, 30)


In [7]:
training_feat

array([[21.47218478, 90.04014005, 11.05126661, ..., 84.26812396,
         3.69941841,  9.40658226],
       [34.59418053, 86.13075664, 59.7060796 , ...,  4.05397443,
        14.66966508,  7.93387018],
       [13.52182056, 50.00514596, 79.03533638, ..., 32.93895487,
        11.08870387, 30.80815178],
       ...,
       [24.15637886, 34.66605913, 20.25588829, ..., 18.97459701,
        37.61646917, 29.57489185],
       [ 4.50644774, 23.09436806, 44.70916402, ...,  1.02775268,
        25.50172331, 15.28646073],
       [12.28109588, 64.9850014 , 32.62234029, ..., 71.40155902,
         5.48593596, 18.74249612]])

## Preprocessing the Data (5 points)

You should explore the data and figure out the best way to preprocess the data. 

Hints: 
1. For the regression values, these at the end will represent colors in RGB space from [0,1]. It is recommended to use a max-min scalar between 0 and 1. 
2. For the training features, you should look at the data and determine the best scaling method. Look at our class notes for a reminder of what other scaler might be useful. 

In [8]:
pd_training_feat = pd.DataFrame(training_feat)
pd_training_feat.describe()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,...,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,19.782907,46.090433,42.772808,8.126919,39.682211,12.045670,43.537720,17.445476,19.697835,6.843953,...,25.922545,0.877013,39.781936,34.854689,25.343963,39.631969,39.267342,42.369058,20.196017,16.605788
std,11.420306,26.538798,24.760044,4.705844,22.940839,6.945770,25.129360,10.080399,11.357079,3.952258,...,14.982179,0.504640,23.023977,20.194849,14.562570,22.898446,22.717096,24.526511,11.692278,9.597830
min,0.001579,0.002765,0.001114,0.000056,0.000181,0.000061,0.000499,0.000211,0.000515,0.000537,...,0.000710,0.000038,0.000355,0.000238,0.000125,0.000052,0.000861,0.000649,0.000595,0.000657
25%,9.876810,23.044580,21.399968,4.045547,19.923214,6.029313,21.778948,8.728659,9.850056,3.413296,...,12.960159,0.439992,19.848774,17.352205,12.804115,19.777779,19.559237,20.982007,10.065367,8.294438
50%,19.795351,46.160818,42.720351,8.144240,39.705752,12.034328,43.697342,17.441851,19.683303,6.848590,...,25.892752,0.880012,39.701756,34.781714,25.306480,39.785349,39.263032,42.506807,20.228040,16.607866
75%,29.667357,68.986637,64.320598,12.206382,59.435630,18.062989,65.321451,26.201945,29.539296,10.273923,...,38.952819,1.314513,59.760552,52.315396,37.957108,59.311149,58.769058,63.618503,30.304817,24.919081
max,39.620228,92.085579,85.570883,16.266999,79.383920,24.076289,87.050661,34.884663,39.357229,13.685279,...,51.827134,1.752801,79.734872,69.882915,50.577111,79.361110,78.689761,84.782527,40.442033,33.241679


In [9]:
pd_training_true = pd.DataFrame(training_true)
pd_training_true.describe()

,0,1,2
count,100000.000000,100000.000000,100000.000000
mean,0.488111,0.492417,0.493129
std,0.156564,0.154757,0.158468
min,0.000000,0.000000,0.000000
25%,0.376602,0.383181,0.380438
50%,0.488199,0.491776,0.493241
75%,0.599458,0.602343,0.605946
max,1.000000,1.000000,1.000000


In [10]:
pd_validation_feat = pd.DataFrame(validation_feat)
pd_validation_feat.describe()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
count,65536.000000,65536.000000,65536.000000,65536.000000,65536.000000,65536.000000,65536.000000,65536.000000,65536.000000,65536.000000,...,65536.000000,65536.000000,65536.000000,65536.000000,65536.000000,65536.000000,65536.000000,65536.000000,65536.000000,65536.000000
mean,3.454462,74.067802,44.857502,10.751527,53.081473,9.529815,17.654575,24.407069,5.338197,6.033054,...,37.387017,0.397624,43.320919,6.054117,39.017395,43.488805,27.581707,28.857502,27.298527,26.303207
std,7.153041,17.097353,7.521236,2.300407,12.401176,3.031889,16.382879,6.787110,4.960646,1.113375,...,8.396434,0.325775,6.768171,10.282923,7.429106,11.945509,7.738622,7.772390,4.468032,4.559016
min,0.205145,0.140010,0.007704,0.106029,0.826440,0.006542,0.116289,0.019159,0.054980,0.020258,...,0.086026,0.012425,0.079528,0.141013,0.534217,0.136502,0.251137,1.260045,0.274257,0.086213
25%,0.671194,80.687513,46.080703,11.513736,57.172443,8.511868,11.549045,26.954927,4.106729,6.053575,...,40.397061,0.275670,44.725183,2.465425,41.724439,41.317566,25.968841,27.969908,27.371512,27.943945
50%,0.671194,80.687513,46.080703,11.513736,57.172443,8.511868,11.549045,26.954927,4.106729,6.053575,...,40.397061,0.275670,44.725183,2.465425,41.724439,41.317566,25.968841,27.969908,27.371512,27.943945
75%,0.671194,80.687513,46.080703,11.513736,57.172443,8.511868,11.549045,26.954927,4.106729,6.053575,...,40.397061,0.275670,44.725183,2.465425,41.724439,41.317566,25.968841,27.969908,27.371512,27.943945
max,39.270873,91.960012,85.403592,16.114298,79.153604,23.595508,86.511252,34.759464,38.897368,13.670417,...,50.860761,1.743625,79.640676,69.227956,50.451757,77.844461,78.319494,84.625035,40.362388,33.192081


In [11]:
train_feat_scaled = StandardScaler().fit_transform(training_feat)
#train_feat_scaled = MinMaxScaler().fit_transform(training_feat)
train_true_scaled = MinMaxScaler().fit_transform(training_true)
validation_feat_scaled = StandardScaler().fit_transform(validation_feat)
#validation_feat_scaled = MinMaxScaler().fit_transform(validation_feat)

In [12]:
pd_train_feat_scaled = pd.DataFrame(train_feat_scaled)
pd_train_feat_scaled.describe()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
count,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,...,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05
mean,3.653100e-16,-6.528778e-17,4.543882e-16,-8.614331e-17,-3.015832e-16,-2.353073e-16,-5.550449e-17,-4.165557e-18,-6.577405e-17,9.258816e-17,...,-2.071834e-16,-2.781519e-16,-4.368872e-16,1.129030e-16,-2.371281e-16,-8.577583e-17,-2.136802e-16,-3.516409e-16,1.580336e-16,-2.420630e-16
std,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,...,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00
min,-1.732128e+00,-1.736623e+00,-1.727457e+00,-1.726981e+00,-1.729764e+00,-1.734245e+00,-1.732533e+00,-1.730621e+00,-1.734374e+00,-1.731529e+00,...,-1.730187e+00,-1.737831e+00,-1.727841e+00,-1.725917e+00,-1.740350e+00,-1.730778e+00,-1.728508e+00,-1.727462e+00,-1.727253e+00,-1.730101e+00
25%,-8.674152e-01,-8.683878e-01,-8.632031e-01,-8.673030e-01,-8.613066e-01,-8.661944e-01,-8.658749e-01,-8.647337e-01,-8.671093e-01,-8.680289e-01,...,-8.651913e-01,-8.660091e-01,-8.657610e-01,-8.666850e-01,-8.611056e-01,-8.670584e-01,-8.675494e-01,-8.720016e-01,-8.664437e-01,-8.659657e-01
50%,1.089687e-03,2.652156e-03,-2.118596e-03,3.680685e-03,1.026158e-03,-1.632950e-03,6.352037e-03,-3.596176e-04,-1.279599e-03,1.173276e-03,...,-1.988549e-03,5.943499e-03,-3.482499e-03,-3.613576e-03,-2.573903e-03,6.698311e-03,-1.897305e-04,5.616360e-03,2.738790e-03,2.164662e-04
75%,8.655197e-01,8.627489e-01,8.702690e-01,8.668973e-01,8.610634e-01,8.663328e-01,8.668680e-01,8.686673e-01,8.665529e-01,8.678550e-01,...,8.697225e-01,8.669575e-01,8.677352e-01,8.646162e-01,8.661389e-01,8.594154e-01,8.584642e-01,8.663912e-01,8.645750e-01,8.661680e-01
max,1.737031e+00,1.733137e+00,1.728522e+00,1.729790e+00,1.730621e+00,1.732087e+00,1.731567e+00,1.730018e+00,1.731034e+00,1.731000e+00,...,1.729035e+00,1.735479e+00,1.735284e+00,1.734522e+00,1.732749e+00,1.735023e+00,1.735372e+00,1.729299e+00,1.731580e+00,1.733306e+00


## Building the Dataset (5 points)

When training neural networks it is important to build a dataset that allows the machinery to sample the data. This also can be used to conduct some preprocessing of the data to make it work with PyTorch. 

I have provided you with the framework for a Dataset Class. 

You should:
1. Convert the x and y data to a tensor 'float32' and put it on the GPU.
2. Save the len of the data
3. Add the code so when `__getitem__` is called it returns the x and y values
3. make it so `__len__` returns the lenght when calle


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [14]:
class Data(Dataset):
  '''Dataset Class to store the samples and their corresponding labels, 
  and DataLoader wraps an iterable around the Dataset to enable easy access to the samples.
  '''

  def __init__(self, X: np.ndarray, y: np.ndarray, device = 'cpu') -> None:

    # need to convert float64 to float32 else 
    # will get the following error
    # RuntimeError: expected scalar type Double but found Float
    X = np.float32(X)
    self.X = torch.from_numpy(X).to(device)
    y = np.float32(y)
    self.y = torch.from_numpy(y).to(device)
    self.len = self.X.shape[0]
  
  def __getitem__(self, index: int) -> tuple:
    return self.X[index], self.y[index]

  def __len__(self) -> int:
    return self.len

## Train-test Split (3 points)

1. You should conduct a train-test split of the training data so you can make sure that your model does not overfit the data. A good ratio is 66/33 train 
2. You should instantiate the training dataset using the data class implemented above.


In [15]:
X_train, X_test, y_train, y_test=train_test_split(train_feat_scaled, train_true_scaled,test_size=0.33)

training_data = Data(X_train,y_train)
testing_data= Data(X_test, y_test)

## Build the Dataloader (3 points)

Pytorch uses DataLoaders to efficiently sample from a training dataset. Instantiate a Pytorch DataLoader using the dataset. 

You should set the following parameters:
1. Batch size = 64
2. Shuffle = True

In [16]:
batch_size=64

In [17]:
train_dataloader = DataLoader(training_data, batch_size=64,shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=64,shuffle=True)

## Building a Neural Network (5 points)

Using the provided class framework which inherits the `nn.Module` type in PyTorch builds a 4-layer neural network to complete the multiple regression.



In [18]:
from torch import nn
import torch.nn.functional as F

class Neural_Network(nn.Module):
    ''' Regression Model
    ''' 

  # note, you can ignore the `:int` and `-> None` this is just more advanced doctring syntax
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int) -> None:

        super(Neural_Network, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
            )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
      # In this part you should build a model that returns the 3 outputs of the regression
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

## Instantiate the Model (3 points)



In [19]:
# number of features (len of X cols)
input_dim = X_train.shape[1]
# number of hidden layers set this to 50
hidden_layers = 50
# Add the number of output dimensions
output_dim = 3

In [20]:
# initiate the regression model
# make sure to put it on your GPU
model = Neural_Network(input_dim,hidden_layers,output_dim).to(device)
print(model)

# criterion to computes the loss between input and target
# Choose a good criteria
loss_fn = nn.MSELoss()

# optimizer that will be used to update weights and biases
# you can choose any optimizer. I would recommend ADAM.
# This problem should not be hard to optimize. A good starting learning rate is 3e-5. 
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)

Neural_Network(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=30, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): ReLU()
    (6): Linear(in_features=50, out_features=3, bias=True)
  )
)


In [21]:
print(f"Model structure: {model}\n\n")

Model structure: Neural_Network(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=30, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): ReLU()
    (6): Linear(in_features=50, out_features=3, bias=True)
  )
)




## Train the Model (5 points)

Training the model is conducted in a number of steps using loops.

1. Set up a loop for each epoch
2. Set a parameter to save the running loss
3. Set up a nested loop that goes through the batches from the DataLoader you built
    - I would recommend using enumerate to include the counts in the loop
    - The dataloader will return a tuple that is the inputs and the labels
4. Conduct the forward propagation of the model
    - Give the model the inputs and compute the outputs
    - Compute the loss given the criteria. 
5. Use the zero gradient method to remove the gradients from the optimizer
6. Use the backward method to compute the gradients
7. Use the step method in the optimizer to take an optimization step
8. Compute the running loss by calling the item method and adding it to the running loss for each minibatch
9. For each epoch print the epoch and the loss

Note: If you find this challenging I would recommend that you look at examples of other pytorch training loops online. This is a very standard workflow. 

In [22]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
    test_loss /= num_batches
    correct /= size
    print(f"Avg loss: {test_loss:>8f} \n")

In [ ]:
from torch.utils.data import Dataset, TensorDataset

batch_size=64
train_combined = TensorDataset(torch.stack([torch.Tensor(n) for n in X_train]), torch.stack([torch.Tensor(n) for n in y_train]))
train_dataloader1 = DataLoader(train_combined, batch_size=batch_size, shuffle=True)
test_combined = TensorDataset(torch.stack([torch.Tensor(n) for n in X_test]), torch.stack([torch.Tensor(n) for n in y_test]))
test_dataloader1 = DataLoader(test_combined, batch_size=batch_size, shuffle=True)

In [ ]:
# start training
epochs = 20 # sets the number of epochs to train 20 should be sufficent.
# This should take about 5-10 minutes to train.
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    #test(test_dataloader1, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.233890  [    0/67000]
loss: 0.236259  [ 6400/67000]
loss: 0.190895  [12800/67000]
loss: 0.113618  [19200/67000]
loss: 0.060434  [25600/67000]
loss: 0.036948  [32000/67000]
loss: 0.027442  [38400/67000]
loss: 0.026803  [44800/67000]
loss: 0.023960  [51200/67000]
loss: 0.023588  [57600/67000]
loss: 0.024826  [64000/67000]
Epoch 2
-------------------------------
loss: 0.020816  [    0/67000]
loss: 0.020463  [ 6400/67000]
loss: 0.018550  [12800/67000]
loss: 0.016834  [19200/67000]
loss: 0.019281  [25600/67000]
loss: 0.017851  [32000/67000]
loss: 0.015232  [38400/67000]
loss: 0.015201  [44800/67000]
loss: 0.011906  [51200/67000]
loss: 0.015461  [57600/67000]
loss: 0.013117  [64000/67000]
Epoch 3
-------------------------------
loss: 0.013608  [    0/67000]
loss: 0.012718  [ 6400/67000]
loss: 0.012975  [12800/67000]
loss: 0.014201  [19200/67000]
loss: 0.012215  [25600/67000]
loss: 0.011141  [32000/67000]
loss: 0.009778  [38400/67000]
loss: 0.01

## Validate the Model (3 points)

Use the test dataset from the train-test split to make sure your model is not overfitting

1. You can build a dataloader as you did before, this time with the test data.
2. Build a validation loop, which you should use `with torch.no_grad()` to make sure you do not modify the gradients, or weights. This will fix your model. 
3. Instantiate the loss to be 0.
4. Build a similar loop to grab the validation dataset. 
5. Compute the predictions with the model.
6. Compute the loss using your loss criteria.
7. Print the final loss determined.

In [ ]:
epochs = 20 # sets the number of epochs to train 20 should be sufficent.
# This should take about 5-10 minutes to train.
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    #train(train_dataloader1, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

<p style="color:blue"> Question: Is your model overfitting or not? How do you know? (3 points) </p>

Type your response here

## Crack Your Code (3 points)

1. You can build a dataloader as you did before, this time with the validation features to view your code.
2. Build a loop, you should use `with torch.no_grad()` to make sure you do not modify the gradients or weights. This will fix your model. 
3. Compute the predictions of your model. 
    - Make sure you do all the same preprocess, the data has the same datatype, and is on the same device

In [ ]:
# Data class for the validation features
class Data_val(Dataset):
  '''Dataset Class to store the samples and their corresponding labels, 
  and DataLoader wraps an iterable around the Dataset to enable easy access to the samples.
  '''

  def __init__(self, X: np.ndarray, device = 'cpu') -> None:

    # need to convert float64 to float32 else 
    # will get the following error
    # RuntimeError: expected scalar type Double but found Float
    X = np.float32(X)
    self.X = torch.from_numpy(X).to(device)# here
    self.len = self.X.shape[0] # here
  
  def __getitem__(self, index: int) -> tuple:
    return self.X[index] # here

  def __len__(self) -> int:
    return self.len #here

In [ ]:
# Builing dataloader for validation features
validation_data = Data_val(validation_feat_scaled)
val_dataloader = DataLoader(validation_data, batch_size=batch_size, shuffle=False)

In [ ]:
# Computing predictions on validation features
total_preds = torch.tensor([], device=device)

with torch.no_grad():
    for batch, X in enumerate (val_dataloader):
        
        predicted = model(X)
        # inverse transform of the predictions
        total_preds = torch.cat((total_preds, predicted), 0)

## Reveal Your Code (3 points)

Your code is an image. there are (65536, 3) predictions this corresponds to a (256,256,3) RGB image. 
1. Use the detach() method to remove the gradients from the tensor
2. Transfer the tensor back to the 'cpu' if you had it on a GPU
3. Reshape the image into a 256,256,3 array. 
4. Plot your successful result.

In [ ]:
train_true_scaler = StandardScaler()
train_true_scaled2 = train_true_scaler.fit_transform(training_true)

In [ ]:
# detach() to remove the gradients
# cpu() to transfer from GPU to CPU
# Transform to numpy and rescale back. Reshape the image into (256, 256, 3) shape
image = total_preds.cpu().detach().numpy()
image = train_true_scaler.inverse_transform(image)

image = image.reshape((256, 256, 3))

In [ ]:
# Plot the result
import matplotlib.pyplot as plt
plt.imshow(image)